## Face swap - GAN

Paper [SimSwap: An Efficient Framework For High Fidelity Face Swapping](https://arxiv.org/pdf/2106.06340v1.pdf)

## Installing and importing

In [1]:
!pip install numpy==1.23.5

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 17.1/17.1 MB 64.3 MB/s eta 0:00:00
  Attempting uninstall: numpy
    Found existing installation: numpy 1.26.4
    Uninstalling numpy-1.26.4:
      Successfully uninstalled numpy-1.26.4
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
albucore 0.0.19 requires numpy>=1.24.4, but you have numpy 1.23.5 which is incompatible.
albumentations 1.4.20 requires numpy>=1.24.4, but you have numpy 1.23.5 which is incompatible.
bigframes 1.25.0 requires numpy>=1.24.0, but you have numpy 1.23.5 which is incompatible.
chex 0.1.87 requires numpy>=1.24.1, but you have numpy 1.23.5 which is incompatible.
jax 0.4.33 requires numpy>=1.24, but you have numpy 1.23.5 which is incompatible.
jaxlib 0.4.33 requires numpy>=1.24, but you have numpy 1.23.5 which is incompatible.
xarray 2024.10.0 requires numpy>=1.24, but you have numpy 1.23.5 whic

**Important:** restart the session

In [1]:
import cv2
from google.colab.patches import cv2_imshow
from matplotlib import pyplot as plt

In [2]:
!git clone https://github.com/saurabhkumar9901/SimSwap.git

Cloning into 'SimSwap'...
remote: Enumerating objects: 161, done.
remote: Total 161 (delta 0), reused 0 (delta 0), pack-reused 161 (from 1)
Receiving objects: 100% (161/161), 88.87 MiB | 26.98 MiB/s, done.
Resolving deltas: 100% (11/11), done.


In [4]:
%cd SimSwap/

/content/SimSwap


In [5]:
!pip3 install torch torchvision torchaudio --index-url https://download.pytorch.org/whl/cu118

Looking in indexes: https://download.pytorch.org/whl/cu118


In [6]:
!pip install insightface==0.2.1 onnxruntime moviepy > /dev/null
!pip install imageio > /dev/null
!pip install googledrivedownloader > /dev/null

## Loading the pretrained model

In [7]:
!gdown $'1sYbGp0nb34BE7_8ElmkxYf0wdE4jaqFC' -O $'arcface_checkpoint.tar'
!gdown $'1qfOUiZzYSnT0Uma52MW8lGu8riAN0wAb' -O $'checkpoints.zip'
!gdown $'1zm7rMfCehkDONPbS4OMxqcRW0UPEvDKH' -O $'512.zip'
!gdown $'11Wcu8nSrOk4BNNZY7t0QHbF5htGNocov' -O $'79999_iter.pth'
!gdown $'1EDnVcMSkFGEfwp0TdEjcyF55dnGQm8b1' -O $'antelope.zip'

Downloading...
From (original): https://drive.google.com/uc?id=1sYbGp0nb34BE7_8ElmkxYf0wdE4jaqFC
From (redirected): https://drive.google.com/uc?id=1sYbGp0nb34BE7_8ElmkxYf0wdE4jaqFC&confirm=t&uuid=ea18e481-e8b7-4c29-938d-4dc519d21bce
To: /content/SimSwap/arcface_checkpoint.tar
100% 209M/209M [00:05<00:00, 39.1MB/s]
Downloading...
From (original): https://drive.google.com/uc?id=1qfOUiZzYSnT0Uma52MW8lGu8riAN0wAb
From (redirected): https://drive.google.com/uc?id=1qfOUiZzYSnT0Uma52MW8lGu8riAN0wAb&confirm=t&uuid=b2b81e22-f7f8-4863-87f9-a49304768f88
To: /content/SimSwap/checkpoints.zip
100% 256M/256M [00:06<00:00, 41.5MB/s]
Downloading...
From (original): https://drive.google.com/uc?id=1zm7rMfCehkDONPbS4OMxqcRW0UPEvDKH
From (redirected): https://drive.google.com/uc?id=1zm7rMfCehkDONPbS4OMxqcRW0UPEvDKH&confirm=t&uuid=dc2a501c-243c-4aff-afd1-288a7c9063fc
To: /content/SimSwap/512.zip
100% 283M/283M [00:09<00:00, 30.5MB/s]
Downloading...
From: https://drive.google.com/uc?id=11Wcu8nSrOk4BNNZY7t0QH

In [8]:
!unzip ./checkpoints.zip -d ./checkpoints
!unzip 512.zip -d ./checkpoints
!unzip antelope.zip -d ./insightface_func/models/

Archive:  ./checkpoints.zip
   creating: ./checkpoints/people/
  inflating: ./checkpoints/people/iter.txt  
  inflating: ./checkpoints/people/latest_net_D1.pth  
  inflating: ./checkpoints/people/latest_net_D2.pth  
  inflating: ./checkpoints/people/latest_net_G.pth  
  inflating: ./checkpoints/people/loss_log.txt  
  inflating: ./checkpoints/people/opt.txt  
   creating: ./checkpoints/people/web/
   creating: ./checkpoints/people/web/images/
Archive:  512.zip
  inflating: ./checkpoints/512/550000_net_G.pth  
Archive:  antelope.zip
   creating: ./insightface_func/models/antelope/
  inflating: ./insightface_func/models/antelope/glintr100.onnx  
  inflating: ./insightface_func/models/antelope/scrfd_10g_bnkps.onnx  


## Loading the images

1. image A (input - source) - copy the face
2. video B (output - target) - paste the face

In [9]:
!mv /content/image.jpg /content/SimSwap/demo_file/image.jpg
!mv /content/video.mp4 /content/SimSwap/demo_file/video.mp4

In [10]:
img_A = '/content/SimSwap/demo_file/image.jpg'
vid_B = '/content/SimSwap/demo_file/video.mp4'

In [17]:
%mkdir -p /content/SimSwap/arcface_model
%mkdir -p /content/SimSwap/parsing_model/checkpoint

!mv /content/SimSwap/arcface_checkpoint.tar /content/SimSwap/arcface_model
!mv /content/SimSwap/79999_iter.pth /content/SimSwap/parsing_model/checkpoint

mkdir: cannot create directory ‘/content/SimSwap/arcface_model’: File exists
mkdir: cannot create directory ‘/content/SimSwap/parsing_model/checkpoint’: File exists
mv: cannot stat '/content/SimSwap/arcface_checkpoint.tar': No such file or directory
mv: cannot stat '/content/SimSwap/79999_iter.pth': No such file or directory


## Training

In [11]:
results = '/content/SimSwap/results.mp4'

Parameters

`--use_mask` - more realism ([ref](https://github.com/neuralchen/SimSwap/blob/main/test_wholeimage_swapmulti.py#L96))

`--crop_size` - input size that depends on the dimensions of the images

`--Arc_path` - path to the model that calculates the similarity between faces

`--pic_a_path` - path to image A (*input*)

`--pic_b_path` - path to image B (*output*)

`--output_path` - path to the folder where the results will be saved

`--no_simswaplogo` - no watermark in the result

In [18]:
!python test_video_swapsingle.py --use_mask --crop_size 224 --isTrain false --name people \
--Arc_path arcface_model/arcface_checkpoint.tar --pic_a_path {img_A} --video_path {vid_B} \
--output_path {results} --no_simswaplogo

error: XDG_RUNTIME_DIR not set in the environment.
ALSA lib confmisc.c:855:(parse_card) cannot find card '0'
ALSA lib conf.c:5178:(_snd_config_evaluate) function snd_func_card_inum returned error: No such file or directory
ALSA lib confmisc.c:422:(snd_func_concat) error evaluating strings
ALSA lib conf.c:5178:(_snd_config_evaluate) function snd_func_concat returned error: No such file or directory
ALSA lib confmisc.c:1334:(snd_func_refer) error evaluating name
ALSA lib conf.c:5178:(_snd_config_evaluate) function snd_func_refer returned error: No such file or directory
ALSA lib conf.c:5701:(snd_config_expand) Evaluate error: No such file or directory
ALSA lib pcm.c:2664:(snd_pcm_open_noupdate) Unknown PCM default
ALSA lib confmisc.c:855:(parse_card) cannot find card '0'
ALSA lib conf.c:5178:(_snd_config_evaluate) function snd_func_card_inum returned error: No such file or directory
ALSA lib confmisc.c:422:(snd_func_concat) error evaluating strings
ALSA lib conf.c:5178:(_snd_config_evalu

> Comments:

* It is recommended to use input images where the face is centered and sharp, with good quality.

* If the dimensions are different, it is recommended to resize the image to a smaller size.

---

Based on:

* https://github.com/neuralchen/SimSwap

Training a custom model:
* https://github.com/neuralchen/SimSwap/blob/main/train.ipynb